# TP chapitre 2 - Entraînez-vous : sélectionnez le nombre de voisins dans un kNN pour une régression

I just re-use all that I have done :

In [2]:
data_path = "~/Documents/openclassroom/Fomation_ingenieur_ML/data/"

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## sklearn module : 
from sklearn import model_selection 
from sklearn import preprocessing
from sklearn import neighbors


## Preprocess : 

In [ ]:
df = pd.read_csv(data_path+'winequality-red.csv', sep=";")
## DESIGN AND SAMPLING : 
X = df.drop("quality", axis = 1, inplace = False)
y = df.quality
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, train_size=0.7)

## STANDARDIZATION : 
my_normalizer = preprocessing.StandardScaler().fit(X_train)
X_train_std = pd.DataFrame(my_normalizer.transform(X_train), index = X_train.index, columns = X_train.columns)
X_test_std = pd.DataFrame(my_normalizer.transform(X_test), index = X_test.index, columns = X_test.columns)

## Cross Validation with sklearn :

In [4]:
## CROSS VALIDATION :
param_grid = {"n_neighbors" : np.arange(8,14,1)}
nb_folds = 5
CV_reg = model_selection.GridSearchCV(neighbors.KNeighborsRegressor(),
                                      param_grid, cv=nb_folds)
CV_reg.fit(X_train_std, y_train)

## KNN REGRESSOR WITH THE BEST CV n_neighbors
knn_reg = neighbors.KNeighborsRegressor(**CV_reg.best_params_)
knn_reg.fit(X_train_std, y_train)

## PREDICTION : 
y_pred = knn_reg.predict(X_test_std)

## Cross Validation without sklearn CVGridSearch :
#### Annexe functions : 

In [7]:
## EXTRACT X_train, X_test FROM X AND train_index :    
def extract_Xsplitted_data(X, train_index):    
#     train_index = X_train.index
    test_index = [(index not in train_index) for index in X.index]
    X_train = X.loc[train_index]
    X_test = X.loc[test_index]
    return(X_train, X_test)

## STANDARDIZATION : 
def normalize_from_split(X, train_index):
    X_train, X_test = extract_Xsplitted_data(X, train_index)
    
    my_normalizer = preprocessing.StandardScaler().fit(X_train)
    X_train_std = pd.DataFrame(my_normalizer.transform(X_train), index = X_train.index, columns = X_train.columns)
    X_test_std = pd.DataFrame(my_normalizer.transform(X_test), index = X_test.index, columns = X_test.columns)

    return(X_train_std, X_test_std)


In [8]:
## SET ARGUMENTS/PARAMETERS :
param_grid = {"n_neighbors" : np.arange(8,14,1)}
cv = 5

###############################################################
##TODO : toutes les variables "locales" sont préfixées de 'CV_' 

## INITIALIZATION : 
CV_X = X_train
CV_y = y_train 

## SET FOLDS : 
kf = model_selection.StratifiedKFold(n_splits = 5)
CV_split_iterator = kf.split(CV_X, y = CV_y)
## MAP THE DICT OF LIST INTO LIST OF DICT :
param_dirg = model_selection.ParameterGrid(param_grid)

### LOOP ON FOLDS :
for CV_train_range_index, CV_test_range_index in CV_split_iterator : 
    train_index = CV_X.index[CV_train_range_index]
    test_index = CV_X.index[CV_test_range_index] 
    
    ## NORMALIZE :
    CV_X_train, CV_X_test = normalize_from_split(CV_X, train_index)
    ## GET y SPLIT : 
    CV_y_train, CV_y_test = y[train_index], y[test_index]
    
    ### LOOP ON PARAM NAMES (HERE ONLY 1)
    for kwargs in param_dirg :
        knn_reg = neighbors.KNeighborsRegressor(**kwargs)
        ## FIT KNN on X_train AND PREDICT ON X_test :
        knn_reg.fit(CV_X_train,CV_y_train)
        
        R2 = knn_reg.score(CV_X_test, CV_y_test)
    

In [9]:
data = X_train, y_train 
## SET ARGUMENTS/PARAMETERS :
clf_meth = neighbors.KNeighborsRegressor()
cv = 5 # NUMBER OF FOLDS
param_grid = {"n_neighbors" : np.arange(2,15,1)}

def my_CV(data, param_grid, clf_meth) :
    # MAP THE DICT OF LIST INTO LIST OF DICT :
    param_dirg = model_selection.ParameterGrid(param_grid)

    ###############################################################
    ##TODO : toutes les variables "locales" sont préfixées de 'CV_' 

    ## INITIALIZATION : 
    CV_X, CV_y = data 

    index = [("fold"+str(k+1)) for k in range(cv)]
    k=0 # index for saving info
    columns = [("_".join(str(val) for val in kwargs.values())) for kwargs in param_dirg]
    accuracy_df = pd.DataFrame(index = index, columns = columns )

    ## SET FOLDS : 
    kf = model_selection.StratifiedKFold(n_splits = 5)
    CV_split_iterator = kf.split(CV_X, y = CV_y)

    ### LOOP ON FOLDS :
    for CV_train_range_index, CV_test_range_index in CV_split_iterator : 
        k+=1
        index_name = "fold"+str(k)
        train_index = CV_X.index[CV_train_range_index]
        test_index = CV_X.index[CV_test_range_index] 

        ## NORMALIZE :
        CV_X_train, CV_X_test = normalize_from_split(CV_X, train_index)
        ## GET y SPLIT : 
        CV_y_train, CV_y_test = y[train_index], y[test_index]

        ### LOOP ON PARAM NAMES (HERE ONLY 1)
        for kwargs in param_dirg :
            ## FIT KNN on X_train AND PREDICT ON X_test :
            knn_clf = neighbors.KNeighborsClassifier(**kwargs)
            knn_clf.fit(CV_X_train,CV_y_train)


            column_name = "_".join(str(val) for val in kwargs.values())
            accuracy_df.loc[index_name, column_name] = knn_clf.score(CV_X_test, CV_y_test)
            
    return pd.DataFrame((accuracy_df.columns, accuracy_df.mean(axis = 0), accuracy_df.std(axis = 0)), index = ['n_neighbors', 'mean','std'])

res= my_CV(data, param_grid, clf_meth)

for col_index in res.columns:
    mean = res.loc["mean",col_index]
    std = res.loc["std",col_index]
    params = res.loc["n_neighbors",col_index]
    print("accuracy = %0.3f (+/-%0.3f) for n_neighbors = %s" %(mean, 2*std, params))
    print("accuracy = {} (+/-{}) for n_neighbors = {}".format(mean, 2*std, params))


0.3555672511607434